In [ ]:
!pip install pandas 
!pip install numpy 
!pip install scikit-learn 
!pip install matplotlib 
!pip install seaborn

In [1]:
from src.utils import convert_to_category, convert_category_into_integer
from src.model.svm import svc_tuning

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings

In [2]:
## 경고 무시
warnings.filterwarnings('ignore')

## train, test 데이터 불러오기
origin_train = pd.read_csv('../data/train.csv', encoding='cp949').dropna()
origin_test = pd.read_csv('../data/test.csv', encoding='cp949').dropna()
origin_train = origin_train.filter(['Churn', 'CustomerCareCalls', 'MonthsInService', 'DroppedCalls', 'OverageMinutes'])
origin_test = origin_test.filter(['Churn', 'CustomerCareCalls', 'MonthsInService', 'DroppedCalls', 'OverageMinutes'])
origin_train = origin_train.query('OverageMinutes < 50')
train = origin_train
test = origin_test

## 데이터 종류가 object인 칼럼 추려내기
objects = []
train.dtypes
for idx in range(len(train.columns)):
    if train.dtypes[idx] == object:
        objects.append(train.columns[idx])

## 범주형 변수 인코딩하기
train, _ = convert_category_into_integer(train, objects)
test, _ = convert_category_into_integer(test, objects)

## 데이터 종류가 object인 칼럼을 카테고리화 하기
train = convert_to_category(train)
test = convert_to_category(test)

## 타겟 변수 정의
target = 'Churn'

## train 데이터 분할하기
seed = 0
train, valid = train_test_split(train, test_size=0.4, random_state=seed)

## 특성 스케일링
compare_columns = ['CustomerCareCalls', 'MonthsInService', 'DroppedCalls', 'OverageMinutes']
standard_scaler = StandardScaler()
train.loc[:, compare_columns] = standard_scaler.fit_transform(train.loc[:, compare_columns])
valid.loc[:, compare_columns] = standard_scaler.transform(valid.loc[:, compare_columns])

train = pd.DataFrame(train, columns=train.columns)

## SVM
# 이상치 처리에 민감하지 않음
# 복잡한 데이터셋 처리 가능
svc_tuning(target, train, valid, seed, sample_size=300)


svc kernel(poly) best score: 0.76
svc kernel(poly) best hyperparameter: {'gamma': 0.001, 'coef0': 4.7, 'C': 0.52}

svc kernel(rbf) best score: 0.76
svc kernel(rbf) best hyperparameter: {'gamma': 1, 'coef0': 2.0, 'C': 0.13}

